# Convolutional neural networks

## 1. Useful links

- [Convolutional arithmetics](http://deeplearning.net/software/theano_versions/dev/tutorial/conv_arithmetic.html)
- [Stanford's CS231n CNN Tutorial](http://cs231n.github.io/convolutional-networks/)
- [Deep learning book](http://deeplearningbook.org/)
- [Tensorflow/Keras tutorial](https://www.tensorflow.org/guide/keras)
- [Hvass Tensorflow tutorias](https://github.com/Hvass-Labs/TensorFlow-Tutorials)
- [Neural network zoo](http://www.asimovinstitute.org/neural-network-zoo/)

## 2. Examples

### 2.1 Cifar10 - Object classification 

#### Imports

In [ ]:
import os
import tensorflow as tf # importing the entire tensorflow library
from tensorflow import keras # importing only the keras module of tensorflow
import numpy as np # the numeric python library

In [ ]:
import pylab # useful for plots (images, graphs, etc.)
%pylab inline
%load_ext tensorboard # useful for plottings neural networks and monitoring the training

#### Loading the Cifar-10 dataset
1. The dataset is available in the Keras module and already splitted (train and test)
2. We normalize the training images.

In [ ]:
cifar10 = keras.datasets.cifar10
data_cifar = cifar10.load_data() # Loading/downloading the images

In [ ]:
(x_train, y_train),(x_test, y_test) = data_cifar # accessing the splits

# Normalization
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train =  (x_train - x_train.mean(0))/x_train.std(0)

#### Plotting some samples

In [ ]:
fig = plt.figure(figsize=(7,7))
for i in range(1, 17):
    plt.subplot(4, 4, i)
    plt.imshow(x_train[i], cmap=plt.cm.gray, interpolation='bilinear')
    plt.title("True label: " + str(y_train[i]))
    plt.xticks(())
    plt.yticks(())

#### Building the CNN
1. Conv layers
2. MaxPool layers
3. Flattening layer
4. Dense layers

In [ ]:
model = keras.models.Sequential([
  keras.layers.Conv2D(10, (3,3), activation='relu', input_shape=(32, 32, 3)),
  keras.layers.MaxPooling2D(2,2),
  keras.layers.Conv2D(20, (3,3), activation='relu'),
  keras.layers.MaxPooling2D(2,2),
  keras.layers.Conv2D(30, (3,3), activation='relu'),
  keras.layers.Flatten(),
  keras.layers.Dense(100, activation='relu'),
  keras.layers.Dense(10, activation='softmax')
])
model.summary()

#### Optimization algorithm and model compilation

In [ ]:
optim = keras.optimizers.Adam()
model.compile(optimizer=optim,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#### Training

In [ ]:
# define a folder to store the training data for monitoring
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
# give the previous folder to Tensorboard 
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

# Fit/Train the model
model.fit(
    x_train, 
    y_train, 
    epochs=3, # number of iterations
    callbacks=[tensorboard_callback] # functions to call after each iteration (e.g. to compute the stats)
)

#### Testing
1. Normalize the test data
2. Plotting some output

In [ ]:
x_test = (x_test - x_train.mean(0))/x_train.std(0) 

In [ ]:
names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

fig = plt.figure(figsize=(15,15))
for i in range(1, 17):
    plt.subplot(4, 4, i)
    plt.imshow(x_test[i], cmap=plt.cm.gray, interpolation='bilinear')
    pred = model.predict(x_test[i:i+1])
    idx = np.argmax(pred)
    plt.title('True: {}; Pred: {}'.format(names[y_test[i][0]], names[idx]))
    plt.xticks(())
    plt.yticks(())

In [ ]:
%tensorboard --logdir logs